In [1]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,GRU
import math
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
import plotly.express as px
import datetime

In [2]:
reliance_stock_market_data=yf.download(tickers="RELIANCE.NS",start="2015-01-01",end="2023-01-31")
reliance_stock_market_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-01 00:00:00+05:30,438.988831,441.812073,437.676270,439.781311,417.988892,1367720
2015-01-02 00:00:00+05:30,439.880371,443.818054,437.998199,438.617340,416.882629,3383424
2015-01-05 00:00:00+05:30,438.344940,441.267212,432.946106,433.812897,412.316254,4662967
2015-01-06 00:00:00+05:30,430.915344,432.401276,412.093750,414.124512,393.603485,8596809
2015-01-07 00:00:00+05:30,414.619812,425.367950,414.570282,423.139069,402.171326,9562421
...,...,...,...,...,...,...
2023-01-23 00:00:00+05:30,2449.000000,2466.199951,2425.000000,2430.300049,2430.300049,5055324
2023-01-24 00:00:00+05:30,2440.000000,2443.649902,2387.350098,2415.949951,2415.949951,7609558
2023-01-25 00:00:00+05:30,2412.449951,2414.699951,2380.000000,2382.550049,2382.550049,5715684


In [3]:
closing_amount=reliance_stock_market_data["Close"]
closing_amount

Date
2015-01-01 00:00:00+05:30     439.781311
2015-01-02 00:00:00+05:30     438.617340
2015-01-05 00:00:00+05:30     433.812897
2015-01-06 00:00:00+05:30     414.124512
2015-01-07 00:00:00+05:30     423.139069
                                ...     
2023-01-23 00:00:00+05:30    2430.300049
2023-01-24 00:00:00+05:30    2415.949951
2023-01-25 00:00:00+05:30    2382.550049
2023-01-27 00:00:00+05:30    2337.350098
2023-01-30 00:00:00+05:30    2359.750000
Name: Close, Length: 1996, dtype: float64

In [4]:
closing_amount.shape

(1996,)

In [5]:
scaler=MinMaxScaler()
closing_amount_scaled=scaler.fit_transform(np.array(closing_amount).reshape(-1,1))

In [6]:
closing_amount_scaled.shape

(1996, 1)

In [7]:
train_data_length=math.ceil(len(closing_amount_scaled)*0.85)
test_data_length=len(closing_amount_scaled)-train_data_length
train_data_length,test_data_length

(1697, 299)

In [8]:
train_data=closing_amount_scaled[:train_data_length,:]
test_data=closing_amount_scaled[train_data_length:len(closing_amount_scaled),:]
train_data.shape,test_data.shape

((1697, 1), (299, 1))

In [9]:
def create_dataset(dataset,time_step=1):
    dataX,dataY=[],[]
    for i in range(len(dataset)-time_step-1):
        a=dataset[i:(i+time_step),0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX),np.array(dataY)

In [10]:
time_step=30
x_train,y_train=create_dataset(train_data,time_step)
x_test,y_test=create_dataset(test_data,time_step)

In [11]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((1666, 30), (1666,), (268, 30), (268,))

In [12]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)
x_train.shape,x_test.shape

((1666, 30, 1), (268, 30, 1))

In [13]:
model=Sequential()
model.add(LSTM(units=128,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.1))
model.add(LSTM(units=128,return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=128))
model.add(Dense(1))

In [14]:
model.compile(optimizer="adam",loss="mean_squared_error")

In [15]:
x_train.shape,y_train.shape

((1666, 30, 1), (1666,))

In [ ]:
model.fit(x=x_train,y=y_train,batch_size=50,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
34/34 [==============================] - 28s 352ms/step - loss: 0.0135 - val_loss: 0.0047
Epoch 2/10
34/34 [==============================] - 9s 258ms/step - loss: 0.0011 - val_loss: 0.0018
Epoch 3/10
34/34 [==============================] - 9s 260ms/step - loss: 8.0927e-04 - val_loss: 0.0018
Epoch 4/10
34/34 [==============================] - 9s 252ms/step - loss: 8.0771e-04 - val_loss: 0.0018
Epoch 5/10
34/34 [==============================] - 8s 249ms/step - loss: 7.3226e-04 - val_loss: 0.0026
Epoch 6/10
34/34 [==============================] - 9s 272ms/step - loss: 8.4323e-04 - val_loss: 0.0022
Epoch 7/10
34/34 [==============================] - 9s 272ms/step - loss: 8.7551e-04 - val_loss: 0.0017
Epoch 8/10
34/34 [==============================] - 9s 266ms/step - loss: 7.1355e-04 - val_loss: 0.0017
Epoch 9/10
34/34 [==============================] - 9s 267ms/step - loss: 7.1724e-04 - val_loss: 0.0018
Epoch 10/10
21/34 [=================>............] - ETA: 3s - loss: 7.

In [ ]:
pred_train=model.predict(x_train)
pred_test=model.predict(x_test)
pred_train.shape,pred_test.shape

In [ ]:
pred_train=scaler.inverse_transform(pred_train)
pred_test=scaler.inverse_transform(pred_test)
pred_train.shape,pred_test.shape 

In [ ]:
pred_train

In [ ]:
y_train_actual=scaler.inverse_transform(y_train.reshape(-1,1))
y_test_actual=scaler.inverse_transform(y_test.reshape(-1,1))
y_train_actual.shape,y_test_actual.shape

In [ ]:
y_train_actual

In [ ]:
rmse_train=math.sqrt(mean_squared_error(y_train_actual,pred_train))
rmse_test=math.sqrt(mean_squared_error(y_test_actual,pred_test))
rmse_train,rmse_test

In [ ]:
input_data = test_data[len(test_data)-time_step:].reshape(1,-1)

temp_input_data = input_data.tolist()

temp_input_data = temp_input_data[0]

output_data=[]

i = 0

prediction_days = 30

while(i<prediction_days):
    
    if(len(temp_input_data)>time_step):
        input_data=np.array(temp_input_data[1:])
        input_data=input_data.reshape(1,-1)
        input_data=input_data.reshape(1,time_step,1)
        predicted_data=model.predict(input_data,verbose=0)
        temp_input_data.extend(predicted_data[0].tolist())
        temp_input_data=temp_input_data[1:]
        output_data.extend(predicted_data.tolist())
        i+=1
    else:
        input_data=input_data.reshape(1,time_step,1)
        predicted_data=model.predict(input_data,verbose=0)
        temp_input_data.extend(predicted_data[0].tolist())
        output_data.extend(predicted_data.tolist())
        i+=1
        
print(len(output_data))

In [ ]:
future_data=scaler.inverse_transform(output_data).reshape(1,-1).tolist()[0]

In [ ]:
future_days_closing_price=pd.DataFrame({"Predictions":future_data})
future_days_closing_price